In [1]:
from bs4 import BeautifulSoup as bsoup
import requests
import sys
from requests.exceptions import RequestException
from contextlib import closing
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import WebDriverException
import re
import os
import json
import time
import random

In [2]:
def simple_get(url,sess):
    try:
        with closing(sess.get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None
    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None

def is_good_response(resp):
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)

def log_error(e):
    print(e)

In [3]:
def total_page(url, sess):
    cont=simple_get(url, sess)
    soup = bsoup(cont, 'html.parser')
    homepage=soup.find("div",{'id':'homepage'})
    page=int(homepage.find("span",{"class":"paginate"}).text.split()[-1])
    return page

In [4]:
def data_extract(url, sess):
    global iitm
    deg = re.compile("PhD.*|M.Tech.*|MS|Postdoctoral|B.Tech|B.Tech(DD)",re.IGNORECASE)
    constraint=re.compile("Advisor|Next|Research|Guide|Faculty|Thesis",re.IGNORECASE)
    cont=simple_get(url, sess)
    soup = bsoup(cont, 'html.parser')
    homepage=soup.find("div",{'id':'homepage'})
    #page=int(homepage.find("span",{"class":"paginate"}).text.split()[-1])
    table=homepage.find('table')
    rows=table.find_all('tr')
    for row in rows:
        td=row.find_all('td')
        for i in td:
            sch={}
            i=str(i).split('<br/>')
            for j in i:
                b=re.sub(r"<[^>]*>", "", j).strip()
                if b =='':
                    continue
                elif ":" in b and constraint.match(b):
                    tmp=b.split(":")
                    sch[tmp[0]]=":".join(tmp[1:])
                elif deg.match(b) is not None:
                        sch['Degree']=b
                else:
                    if hasNumbers(b) or constraint.match(b):
                        print(b)
                        print('somthing wrong with name field')
                    else:
                        if sch.get('Name'): 
                            sch['Name']=sch.get('Name')+"@"+b
                        else:
                            sch['Name']=b
            if len(sch) != 0:
                iitm.append(sch)
    return

In [5]:
def write_file(data, filename, location='./college/'):
    if not os.path.exists(location):
        try:
            os.makedirs(location)
        except OSError as e:
            if e.errno != errno.EEXIST:
                raise
    path=os.path.join(location, filename + "." + 'json')
    with open(path,'w') as f:
        json.dump(data, f, sort_keys=True, indent=4)
    print('Done')
    return

In [6]:
def hasNumbers(inputString):
    return bool(re.search(r'\d', inputString))

In [7]:
%%time
#mtech="U3R1ZGVudHMhMTMkMSQk"
#btech_dd="U3R1ZGVudHMhMTQkMSQk"
#btech="U3R1ZGVudHMhMTUkMSQk"
filename='iitm'
args=["U3R1ZGVudHMhMTUkMSQk","U3R1ZGVudHMhMTQkMSQk","U3R1ZGVudHMhMTMkMSQk"]
iitm=[]
url='http://www.cse.iitm.ac.in/listpeople.php?arg=MyQxJCQ=&%20page=1&ipp=50'
tmp_url='http://www.cse.iitm.ac.in/listpeople.php?arg=MyQxJCQ=&%20page=1&ipp=50'
with requests.Session() as sess:
    for arg in range(len(args)+1):
        page=total_page(url, sess)
        for i in range(2, page+2):
            data_extract(url, sess)
            url=url.split('page')[0]+"page="+str(i)+"&ipp=50"
            time.sleep(random.randint(10,15))
        if arg < len(args):
            tmp_url1=tmp_url.split('MyQxJCQ')
            url=tmp_url1[0]+args[arg]+tmp_url1[1]
write_file(iitm, filename)

Anirban was a part-time PhD scholar (from IBM Research). He discontinued from the program as on Jan 2018.
somthing wrong with name field
Anirban was a part-time PhD scholar (from IBM Research). He discontinued from the program as on Jan 2018.
somthing wrong with name field
Done
CPU times: user 1.96 s, sys: 20.4 ms, total: 1.98 s
Wall time: 5min 38s


In [32]:
iitm=[]
#args=["U3R1ZGVudHMhMTUkMSQk","U3R1ZGVudHMhMTQkMSQk","U3R1ZGVudHMhMTMkMSQk"]
args=[]
url='http://www.cse.iitm.ac.in/listpeople.php?arg=MyQxJCQ=&%20page=1&ipp=50'
#tmp_url='http://www.cse.iitm.ac.in/listpeople.php?arg=MyQxJCQ=&%20page=1&ipp=50'
with requests.Session() as sess:
    for arg in range(len(args)+1):
        #page=total_page(url, sess)
        for i in range(1, 2):
            data_extract(url, sess)
            url=url.split('page')[0]+"page="+str(i)+"&ipp=50"
            time.sleep(random.randint(10,15))
        if arg < len(args):
            tmp_url1=tmp_url.split('MyQxJCQ')
            url=tmp_url1[0]+args[arg]+tmp_url1[1]


Anirban was a part-time PhD scholar (from IBM Research). He discontinued from the program as on Jan 2018.
somthing wrong with name field


In [8]:
len(iitm)

1300

In [65]:
write_file(iitm,'iitm')

Done


In [10]:
for i in range(2,6):
    print(i)

2
3
4
5


In [14]:
len(iitm1)

250

In [ ]:
from scrapely import Scraper
s = Scraper()

In [56]:
a={u'Name': 'Ramesh .S',
 u'Degree': 'MS Scholar,  Jul 1999 -  Jan 2002',
 u'Advisor(s)': 'Sukhendu Das',
 u'Thesis Title': ' Edginess Image for Face Recognition'}